# Equal Weighted S&P500 Index Fund Algorithmic Trading Strategy
- The following represents an algorithmic trading strategy that involves building a portfolio that has an equal magnitude for each stock in the S&P500 index fund, as opposed to a spread based on market capitalisation
- This notebook does not constitute financial advice, and is merely the educational exploration of the field of algorithmic trading
- The code contains very minor refactoring, but is majoritively based off of [this notebook](https://github.com/nickmccullum/algorithmic-trading-python/blob/master/finished_files/001_equal_weight_S%26P_500.ipynb) from [this tutorial](https://www.youtube.com/watch?v=xfzGZB4HhEE)

In [ ]:
import pandas as pd
import requests
import math
from secrets import IEX_CLOUD_API_TOKEN # Secrets file not included in repository

## 1. Batch call the IEX endpoint to retrieve the data
5 calls for 100 tickers each

In [ ]:
my_columns = ['Ticker', 'Price','Market Capitalization', 'Number Of Shares to Buy']
stocks = pd.read_csv('sp_500_stocks.csv') # Not included in repository

# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'], 
                                                   data[symbol]['quote']['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)

## 2. Calculate the number of shares to buy of each stock

In [ ]:
portfolio_size = input("Enter the value of your portfolio:")

while True:
    try:
        portfolio_size = float(input("Enter the value of your portfolio: "))
        break
    except ValueError:
        print("That's not a number! Please enter a valid number.")

position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])-1):
    final_dataframe.loc[i, 'Number Of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

## 3. Format the output as an Excel spreadsheet

In [ ]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, sheet_name='Recommended Trades', index = False)

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [ ]:
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

In [ ]:
writer.save()